<a href="https://colab.research.google.com/github/nedokormysh/GB_intro_Analysis_Healthcare/blob/lesson_5_personal/GB_intro_Healthcare_Analysis_hw_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для некоторых задач возможно выбрать разные модели, но часть задач для них работают только специфичные модели. Изначально хотел рассматривать задачу всо сном, но там может работать только одна модель. Поэтому решил рассматривать данную задачу.

Изучите https://pyhealth.readthedocs.io/en/latest/index.html

В разделе Tutorials выберите наиболее интересующий вас пайплайн (например, Mortality Prediction) и попробуйте его прогнать на предложенных разработчиками датасетах (графа Datasets) стартовой страницы.

Выберите модель, подходящую для вашей задачи, аргументируйте, почему именно эта модель кажется вам наилучшей. Обучите модель и протестируйте ее, оцените качество.

# **Прогнозирование повторной госпитализации**

### **Установка библиотеки**

In [ ]:
!pip install pyhealth -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.3/226.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 42.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.5 MB/s eta 0:00:00


### **Загрузка датасета**

Хотел использовать данные с MIMIC. Но как получить данные - так и не понял.

Тот рут, который они приводят на сайте как, которым можно воспользоваться - не работает, либо я не понял как перейти на этот сервер.

Например:

* Для mimic 3 root="/srv/local/data/physionet.org/files/mimiciii/1.4"
* Для mimic 4 root="/srv/local/data/physionet.org/files/mimiciv/2.0/hosp"

Но доступ по этим ссылкам закрыт. Т.е. можно физически скачать таблицы из демо версии, например (https://physionet.org/files/mimiciii-demo/1.4/). Но просто поставить сслыку на хотя бы демо-версию, и ожидать, что программа отработает и вытащит файлы, которые нужны, как они говорят в туториалах - так не работает.

Поэтому запустил только тот синтетический датасет, который они сами предоставили.

In [ ]:
from pyhealth.datasets import MIMIC3Dataset

mimic3_ds = MIMIC3Dataset(
        root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
        tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
)

mimic3_ds.stat()

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
finish basic patient information parsing : 96.79241251945496s
finish parsing DIAGNOSES_ICD : 68.34779691696167s
finish parsing PROCEDURES_ICD : 55.983074426651s
finish parsing PRESCRIPTIONS : 175.3561692237854s


Mapping codes: 100%|██████████| 49993/49993 [00:01<00:00, 33009.51it/s]



Statistics of base dataset (dev=False):
	- Dataset: MIMIC3Dataset
	- Number of patients: 49993
	- Number of visits: 52769
	- Number of visits per patient: 1.0555
	- Number of events per visit in DIAGNOSES_ICD: 9.1038
	- Number of events per visit in PROCEDURES_ICD: 3.2186
	- Number of events per visit in PRESCRIPTIONS: 25.9600



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 49993\n\t- Number of visits: 52769\n\t- Number of visits per patient: 1.0555\n\t- Number of events per visit in DIAGNOSES_ICD: 9.1038\n\t- Number of events per visit in PROCEDURES_ICD: 3.2186\n\t- Number of events per visit in PRESCRIPTIONS: 25.9600\n'

In [ ]:
# data format
mimic3_ds.info()


dataset.patients: patient_id -> <Patient>

<Patient>
    - visits: visit_id -> <Visit> 
    - other patient-level info
    
    <Visit>
        - event_list_dict: table_name -> List[Event]
        - other visit-level info
    
        <Event>
            - code: str
            - other event-level info



Т.е. у нас есть пациент, у него есть визиты в больницу, и у этих визитов есть события: диагнозы, лабораторные тесты,...

### Определим функцию для передачи определённого пациента из нашего датасета

In [ ]:
from pyhealth.tasks import readmission_prediction_mimic3_fn

mimic3_ds = mimic3_ds.set_task(task_fn=readmission_prediction_mimic3_fn)

mimic3_ds.stat()

Generating samples for readmission_prediction_mimic3_fn: 100%|██████████| 49993/49993 [00:00<00:00, 347936.78it/s]


Statistics of sample dataset:
	- Dataset: MIMIC3Dataset
	- Task: readmission_prediction_mimic3_fn
	- Number of samples: 2194
	- Number of patients: 2116
	- Number of visits: 2194
	- Number of visits per patient: 1.0369
	- conditions:
		- Number of conditions per sample: 10.3414
		- Number of unique conditions: 2526
		- Distribution of conditions (Top-10): [('4019', 875), ('41401', 591), ('42731', 540), ('25000', 483), ('4280', 472), ('2724', 324), ('5849', 297), ('53081', 280), ('5990', 264), ('2720', 254)]
	- procedures:
		- Number of procedures per sample: 4.0789
		- Number of unique procedures: 815
		- Distribution of procedures (Top-10): [('3893', 458), ('3961', 419), ('9904', 393), ('8856', 250), ('3615', 234), ('9604', 222), ('966', 213), ('8872', 208), ('9671', 181), ('3722', 177)]
	- drugs:
		- Number of drugs per sample: 35.5228
		- Number of unique drugs: 2424
		- Distribution of drugs (Top-10): [('00338001702', 1023), ('00008084199', 903), ('51079025520', 893), ('00338004938

"Statistics of sample dataset:\n\t- Dataset: MIMIC3Dataset\n\t- Task: readmission_prediction_mimic3_fn\n\t- Number of samples: 2194\n\t- Number of patients: 2116\n\t- Number of visits: 2194\n\t- Number of visits per patient: 1.0369\n\t- conditions:\n\t\t- Number of conditions per sample: 10.3414\n\t\t- Number of unique conditions: 2526\n\t\t- Distribution of conditions (Top-10): [('4019', 875), ('41401', 591), ('42731', 540), ('25000', 483), ('4280', 472), ('2724', 324), ('5849', 297), ('53081', 280), ('5990', 264), ('2720', 254)]\n\t- procedures:\n\t\t- Number of procedures per sample: 4.0789\n\t\t- Number of unique procedures: 815\n\t\t- Distribution of procedures (Top-10): [('3893', 458), ('3961', 419), ('9904', 393), ('8856', 250), ('3615', 234), ('9604', 222), ('966', 213), ('8872', 208), ('9671', 181), ('3722', 177)]\n\t- drugs:\n\t\t- Number of drugs per sample: 35.5228\n\t\t- Number of unique drugs: 2424\n\t\t- Distribution of drugs (Top-10): [('00338001702', 1023), ('000080841

In [ ]:
from pyhealth.datasets.splitter import split_by_patient
from pyhealth.datasets import split_by_patient, get_dataloader

# data split
train_dataset, val_dataset, test_dataset = split_by_patient(mimic3_ds, [0.8, 0.1, 0.1])

# create dataloaders (they are <torch.data.DataLoader> object)
train_loader = get_dataloader(train_dataset, batch_size=64, shuffle=True)
val_loader = get_dataloader(val_dataset, batch_size=64, shuffle=False)
test_loader = get_dataloader(test_dataset, batch_size=64, shuffle=False)

# Модели

Собственно, наши данные, в основном состоят из кодировок(которые по сути, мы не можем сравнивать), и текстовых назначений. Думаю лучше всего с данной задачей должны справиться модели Трансформеры или RNN. Учитывая, что тут не большие последовательности, то RNN возможно будет лучше всего работать.

Но заранее предсказать какая модель будет справляться лучше - не готов. Будем использовать всего 5 эпох обучения и стандартные настройки. Т.е. не будем подбирать параметры сеток. Также в данной библиотеке есть возможность добавлять слои в сетку. Опять же, для простого сравнения, делать этого не будем.

In [ ]:
import pandas as pd

In [ ]:
df_results = pd.DataFrame()

### Transformer

#### Определим ML модель

In [ ]:
from pyhealth.models import Transformer

model = Transformer(
    dataset=mimic3_ds,
    # look up what are available for "feature_keys" and "label_keys" in dataset.samples[0]
    feature_keys=["conditions", "procedures"],
    label_key="label",
    mode="binary",
)

In [ ]:
all_metrics = [
        "pr_auc",
        "roc_auc",
        "accuracy",
        "balanced_accuracy",
        # "f1",
        # "precision",
        # "recall",
        # "cohen_kappa",
        # "jaccard",
    ]

#### Обучение модели

In [ ]:
from pyhealth.trainer import Trainer

trainer = Trainer(model=model, metrics=all_metrics)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor='roc_auc',
)

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
            (activation): GELU(approximate='none')
          )
          (i

/usr/local/lib/python3.10/dist-packages/pyhealth/trainer.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
INFO:pyhealth.trainer:Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
 

Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


INFO:pyhealth.trainer:Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


Device: cpu


...

INFO:pyhealth.trainer:balanced_accuracy: 0.4895


loss: 0.5957


INFO:pyhealth.trainer:loss: 0.5957


Loaded best model


INFO:pyhealth.trainer:Loaded best model


#### Оценка модели

In [ ]:
# option 2: use our pyhealth.metrics to evaluate
from pyhealth.metrics.binary import binary_metrics_fn
y_true, y_prob, loss = trainer.inference(test_loader)
binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

In [ ]:
df_results['Transformer'] = binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

### Retain

Модель, использующая некий механизм внимания.

In [ ]:
from pyhealth.models import RETAIN

model = RETAIN(
    dataset=mimic3_ds,
    # look up what are available for "feature_keys" and "label_keys" in dataset.samples[0]
    feature_keys=["conditions", "procedures"],
    label_key="label",
    mode="binary",
)

In [ ]:
trainer = Trainer(model=model, metrics=all_metrics)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor='roc_auc',
)

RETAIN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (retain): ModuleDict(
    (conditions): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
    (procedures): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


INFO:pyhealth.trainer:RETAIN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (retain): ModuleDict(
    (conditions): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
    (procedures): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


INFO:pyhealth.trainer:Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


Device: cpu


...

INFO:pyhealth.trainer:balanced_accuracy: 0.5000


loss: 0.4706


INFO:pyhealth.trainer:loss: 0.4706


Loaded best model


INFO:pyhealth.trainer:Loaded best model


In [ ]:
y_true, y_prob, loss = trainer.inference(test_loader)
binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

In [ ]:
df_results['Retain'] = binary_metrics_fn(y_true, y_prob, metrics=all_metrics);

### RNN

In [ ]:
from pyhealth.models import RNN

model = RNN(
    dataset=mimic3_ds,
    # look up what are available for "feature_keys" and "label_keys" in dataset.samples[0]
    feature_keys=["conditions", "procedures"],
    label_key="label",
    mode="binary",
)

In [ ]:
trainer = Trainer(model=model, metrics=all_metrics)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor='roc_auc',
)

In [ ]:
y_true, y_prob, loss = trainer.inference(test_loader)
binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 107.75it/s]


{'pr_auc': 0.2063854732862991,
 'roc_auc': 0.5626622172784576,
 'accuracy': 0.8651162790697674,
 'balanced_accuracy': 0.5145532072673341}

In [ ]:
df_results['RNN'] = binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

### CNN

В моём понимании CNN тут должны отрабатывать хуже, т.к. это не изображения.

In [ ]:
from pyhealth.models import CNN

model = CNN(
    dataset=mimic3_ds,
    # look up what are available for "feature_keys" and "label_keys" in dataset.samples[0]
    feature_keys=["conditions", "procedures"],
    label_key="label",
    mode="binary",
)

In [ ]:
trainer = Trainer(model=model, metrics=all_metrics)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor='roc_auc',
)

In [ ]:
y_true, y_prob, loss = trainer.inference(test_loader)
binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 95.65it/s]


{'pr_auc': 0.12502513511921703,
 'roc_auc': 0.45587690025954775,
 'accuracy': 0.8604651162790697,
 'balanced_accuracy': 0.49731182795698925}

In [ ]:
df_results['CNN'] = binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

### Deepr

Вообще это cnn модель для медицинских данных. Со скользящим окном, и векторизацией кодировок

In [ ]:
from pyhealth.models import Deepr

model = Deepr(
    dataset=mimic3_ds,
    # look up what are available for "feature_keys" and "label_keys" in dataset.samples[0]
    feature_keys=["conditions", "procedures"],
    label_key="label",
    mode="binary",
)

In [ ]:
trainer = Trainer(model=model, metrics=all_metrics)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor='roc_auc',
)

Deepr(
  (embeddings): ModuleDict(
    (conditions): Embedding(2529, 128, padding_idx=0)
    (procedures): Embedding(818, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (cnn): ModuleDict(
    (conditions): DeeprLayer(
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    )
    (procedures): DeeprLayer(
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


INFO:pyhealth.trainer:Deepr(
  (embeddings): ModuleDict(
    (conditions): Embedding(2529, 128, padding_idx=0)
    (procedures): Embedding(818, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (cnn): ModuleDict(
    (conditions): DeeprLayer(
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    )
    (procedures): DeeprLayer(
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


INFO:pyhealth.trainer:Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


Device: cpu


...

INFO:pyhealth.trainer:balanced_accuracy: 0.4974


loss: 0.5465


INFO:pyhealth.trainer:loss: 0.5465


Loaded best model


INFO:pyhealth.trainer:Loaded best model


In [ ]:
y_true, y_prob, loss = trainer.inference(test_loader)
binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

In [ ]:
df_results['Deepr'] = binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

# Результаты

In [ ]:
df_results

,Transformer,Retain,RNN,CNN,Deepr
pr_auc,0.192795,0.185067,0.206385,0.125025,0.165912
roc_auc,0.597145,0.600482,0.562662,0.455877,0.504264
accuracy,0.865116,0.865116,0.865116,0.860465,0.865116
balanced_accuracy,0.500000,0.500000,0.514553,0.497312,0.500000


По roc_auc лучше всего справилась модель Retain. Впрочем, в задании не фиксировал random_state, а потому думаю результаты могут отличаться от запуска к запуску.